In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *

import random
import time
import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
from scipy.signal import savgol_filter

# Cargue de Queries y parametros

In [2]:
city = 'BAQ'

In [3]:
df_kams_ctm = get_fresh_query_result("https://internal-redash.federate.frubana.com/",169591,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)
df_kams_ctm[["customer_id"]] = df_kams_ctm[["customer_id"]].astype(int)

200


In [5]:
df_bench = get_fresh_query_result("https://internal-redash.federate.frubana.com/",175456,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [6]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [7]:
#customer_segments = get_fresh_query_result("https://internal-redash.federate.frubana.com/",178794,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

In [8]:
query = """
SELECT DISTINCT
split_part(bs.upc, '-',1) as region_code
--,bs.sku_id
--,split_part(bs.upc, ':',4) as IdProd
,bs.addl_product_id as card_id
--,bs.upc
,bcat2.name as categoria
,bcat.name as subcategoria
,        CASE
            WHEN ((bs.active_end_date IS NULL OR bs.active_end_date > CURRENT_TIMESTAMP) AND (bs2.active_end_date IS NULL OR bs2.active_end_date > CURRENT_TIMESTAMP)) THEN 'on'
            ELSE 'off'
        END as status_frida


from postgres_broadleaf_federate."broadleaf.blc_sku" bs
LEFT join postgres_broadleaf_federate."broadleaf.blc_product" bp  on bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku" bs2 on bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
left join postgres_broadleaf_federate."broadleaf.blc_category_xref" bcx on bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and default_reference = 'true' --Relaciones categorias 
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat on bp.default_category_id = bcat.category_id --Nombre subcategoria
left join postgres_broadleaf_federate."broadleaf.blc_category" bcat2 on bcx.category_id = bcat2.category_id --Nombre categoria

where 
bs.archived = 'N'
and bs.sndbx_id is null
and bp.archived = 'N'
and bs.catalog_disc < 0
and ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))
"""
df_skus = read_connection_data_warehouse.runQuery(query)

In [9]:
query = """
WITH MaxSubmissionDates AS (
    SELECT
        su.card_id,
        MAX(submit_date.full_date) AS max_submission_date
    FROM dpr_sales.fact_sales s
    LEFT JOIN dpr_shared.dim_date submit_date
        ON s.dim_submitted_date = submit_date.date_id
    LEFT JOIN dpr_shared.dim_stock_unit su
        ON s.dim_product = su.product_id
    WHERE submit_date.full_date >= DATE(GETDATE()) - 10
        AND s.dim_status = 1
    GROUP BY su.card_id
)

SELECT
    s.dim_submitted_date,
    su.card_id,
    CASE WHEN su.source_parent_id = 0 THEN su.source_id ELSE su.source_parent_id END AS parent_id,
    SUM((s.product_price * s.product_quantity_x_step_unit)) / NULLIF(SUM(s.product_quantity_x_step_unit), 0) AS gross_price_no_iva,
    SUM((s.product_price * s.product_quantity_x_step_unit) + s.product_tax_iva) / NULLIF(SUM(s.product_quantity_x_step_unit), 0) AS gross_price
FROM dpr_sales.fact_sales s
LEFT JOIN dpr_shared.dim_site site
    ON s.dim_site = site.site_id
LEFT JOIN dpr_shared.dim_date submit_date
    ON s.dim_submitted_date = submit_date.date_id
LEFT JOIN dpr_shared.dim_stock_unit su
    ON s.dim_product = su.product_id
INNER JOIN MaxSubmissionDates msd
    ON su.card_id = msd.card_id
    AND submit_date.full_date = msd.max_submission_date
GROUP BY 1, 2, 3
"""
df_prices = run_read_dwd_query(query)
df_prices[["gross_price_no_iva","gross_price"]] = df_prices[["gross_price_no_iva","gross_price"]].astype(float)

In [10]:
df_prices = pd.merge(df_prices, df_bench[['extracted_id','benchmark']],  how='left', left_on=['parent_id'], right_on = ['extracted_id'])
df_prices.drop(columns=['extracted_id'], inplace=True)

df_prices = df_prices.groupby(['card_id']).agg({'gross_price_no_iva':np.mean,'gross_price':np.mean,'benchmark':np.mean}).reset_index().copy()

In [11]:
query = """
WITH base_skus AS (
    SELECT DISTINCT
        bo.offer_id,
        bo.offer_value/100.00 AS dct,
        bocsx.customer_segment_id,
        REPLACE(SPLIT_PART(SPLIT_PART(boic.order_item_match_rule, '[',2),']',1),'"','') AS skus
            
    FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_customer_offer_xref"       bcox    ON (bo.offer_id = bcox.offer_id  )
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"       btcofx  ON (btcofx.offer_id = bo.offer_id)
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id AND (boic.archived is NULL OR boic.archived ='N'))
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc

    WHERE   
        (bo.end_date >= CURRENT_dATE or bo.end_date IS NULL)
        AND bo.start_date <= CURRENT_DATE
        ---TIME VALIDATIONS
        -------------------
        AND bo.offer_discount_type = 'PERCENT_OFF'
        AND bo.offer_type = 'ORDER_ITEM'
        AND bo.sndbx_id is NULL
        AND (bo.archived is NULL OR bo.archived='N')
        AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
        AND bo.automatically_added = 'true'
        AND (boic.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
        AND (bocsx.sndbx_tier is NULL OR bocsx.sndbx_tier = 999999)
        --AND bocsx.customer_segment_id IS NOT NULL
        AND bs.site_identifier_value = 'BOG'
        AND (bo.max_uses_per_customer <> 1 OR bo.max_uses_per_customer IS NULL)
    ),
    
    tab_unnest AS (
        SELECT 1 AS n UNION ALL
        SELECT 2 AS n UNION ALL
        SELECT 3 AS n UNION ALL
        SELECT 4 AS n UNION ALL
        SELECT 5 AS n UNION ALL
        SELECT 6 AS n UNION ALL
        SELECT 7 AS n UNION ALL
        SELECT 8 AS n UNION ALL
        SELECT 9 AS n UNION ALL
        SELECT 10 AS n UNION ALL
        SELECT 11 AS n UNION ALL
        SELECT 12 AS n UNION ALL
        SELECT 13 AS n UNION ALL
        SELECT 14 AS n UNION ALL
        SELECT 15 AS n UNION ALL
        SELECT 16 AS n UNION ALL
        SELECT 17 AS n UNION ALL
        SELECT 18 AS n UNION ALL
        SELECT 19 AS n UNION ALL
        SELECT 20 AS n UNION ALL
        SELECT 21 AS n UNION ALL
        SELECT 22 AS n UNION ALL
        SELECT 23 AS n UNION ALL
        SELECT 24 AS n UNION ALL
        SELECT 25 AS n UNION ALL
        SELECT 26 AS n UNION ALL
        SELECT 27 AS n UNION ALL
        SELECT 28 AS n UNION ALL
        SELECT 29 AS n UNION ALL
        SELECT 30 AS n UNION ALL
        SELECT 31 AS n UNION ALL
        SELECT 32 AS n UNION ALL
        SELECT 33 AS n UNION ALL
        SELECT 34 AS n UNION ALL
        SELECT 35 AS n UNION ALL
        SELECT 36 AS n UNION ALL
        SELECT 37 AS n UNION ALL
        SELECT 38 AS n UNION ALL
        SELECT 39 AS n UNION ALL
        SELECT 40 AS n UNION ALL
        SELECT 41 AS n UNION ALL
        SELECT 42 AS n UNION ALL
        SELECT 43 AS n UNION ALL
        SELECT 44 AS n UNION ALL
        SELECT 45 AS n UNION ALL
        SELECT 46 AS n UNION ALL
        SELECT 47 AS n UNION ALL
        SELECT 48 AS n UNION ALL
        SELECT 49 AS n UNION ALL
        SELECT 50 AS n UNION ALL
        SELECT 51 AS n UNION ALL
        SELECT 52 AS n UNION ALL
        SELECT 53 AS n UNION ALL
        SELECT 54 AS n UNION ALL
        SELECT 55 AS n UNION ALL
        SELECT 56 AS n UNION ALL
        SELECT 57 AS n UNION ALL
        SELECT 58 AS n UNION ALL
        SELECT 59 AS n UNION ALL
        SELECT 60 AS n UNION ALL
        SELECT 61 AS n UNION ALL
        SELECT 62 AS n UNION ALL
        SELECT 63 AS n UNION ALL
        SELECT 64 AS n UNION ALL
        SELECT 65 AS n UNION ALL
        SELECT 66 AS n UNION ALL
        SELECT 67 AS n UNION ALL
        SELECT 68 AS n UNION ALL
        SELECT 69 AS n UNION ALL
        SELECT 70 AS n UNION ALL
        SELECT 71 AS n UNION ALL
        SELECT 72 AS n UNION ALL
        SELECT 73 AS n UNION ALL
        SELECT 74 AS n UNION ALL
        SELECT 75 AS n UNION ALL
        SELECT 76 AS n UNION ALL
        SELECT 77 AS n UNION ALL
        SELECT 78 AS n UNION ALL
        SELECT 79 AS n UNION ALL
        SELECT 80 AS n UNION ALL
        SELECT 81 AS n UNION ALL
        SELECT 82 AS n UNION ALL
        SELECT 83 AS n UNION ALL
        SELECT 84 AS n UNION ALL
        SELECT 85 AS n UNION ALL
        SELECT 86 AS n UNION ALL
        SELECT 87 AS n UNION ALL
        SELECT 88 AS n UNION ALL
        SELECT 89 AS n UNION ALL
        SELECT 90 AS n UNION ALL
        SELECT 91 AS n UNION ALL
        SELECT 92 AS n UNION ALL
        SELECT 93 AS n UNION ALL
        SELECT 94 AS n UNION ALL
        SELECT 95 AS n UNION ALL
        SELECT 96 AS n UNION ALL
        SELECT 97 AS n UNION ALL
        SELECT 98 AS n UNION ALL
        SELECT 99 AS n UNION ALL
        SELECT 100 AS n UNION ALL
        SELECT 101 AS n UNION ALL
        SELECT 102 AS n UNION ALL
        SELECT 103 AS n UNION ALL
        SELECT 104 AS n UNION ALL
        SELECT 105 AS n UNION ALL
        SELECT 106 AS n UNION ALL
        SELECT 107 AS n UNION ALL
        SELECT 108 AS n UNION ALL
        SELECT 109 AS n UNION ALL
        SELECT 110 AS n UNION ALL
        SELECT 111 AS n UNION ALL
        SELECT 112 AS n UNION ALL
        SELECT 113 AS n UNION ALL
        SELECT 114 AS n UNION ALL
        SELECT 115 AS n UNION ALL
        SELECT 116 AS n UNION ALL
        SELECT 117 AS n UNION ALL
        SELECT 118 AS n UNION ALL
        SELECT 119 AS n UNION ALL
        SELECT 120 AS n
    ),
     
    info_skus AS (  
    SELECT 
        bs.offer_id,
        bs.dct,
        bs.customer_segment_id,
        TRIM(SPLIT_PART(bs.skus, ',', tab_unnest.n)) AS sku
        
    FROM tab_unnest
    INNER JOIN base_skus bs ON tab_unnest.n <= REGEXP_COUNT(bs.skus, ',') + 1
    )
    
SELECT DISTINCT
    s.site_identifier_value as ciudad,
    bcs.external_identifier::int AS customer_id,
    bs.addl_product_id AS card_id,
    MAX(bo.dct) AS dct
    
FROM info_skus bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bo.sku
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id AND bo.customer_segment_id IS NOT NULL AND bcs.external_identifier ~ '^[0-9]+$'
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON s.site_id = bs.catalog_disc

GROUP BY 1,2,3
"""
df_filt_offers = read_connection_data_warehouse.runQuery(query)
df_filt_offers[["dct"]] = df_filt_offers[["dct"]].astype(float)

In [12]:
query = """
WITH base_skus AS (
    SELECT DISTINCT
        bo.offer_id,
        bo.offer_value/100.00 AS dct,
        bocsx.customer_segment_id,
        REPLACE(SPLIT_PART(SPLIT_PART(boic.order_item_match_rule, '[',2),']',1),'"','') AS skus
            
    FROM postgres_broadleaf_federate."broadleaf.blc_offer"                          bo
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_offer_customer_seg_xref"   bocsx   ON (bo.offer_id = bocsx.offer_id AND ((bocsx.archived is NULL OR bocsx.archived='N')))
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_customer_offer_xref"       bcox    ON (bo.offer_id = bcox.offer_id  )
    LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_tar_crit_offer_xref"       btcofx  ON (btcofx.offer_id = bo.offer_id)
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_offer_item_criteria"      boic    ON (btcofx.offer_item_criteria_id = boic.offer_item_criteria_id AND (boic.archived is NULL OR boic.archived ='N'))
    INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     bs      ON bs.site_id = bo.catalog_disc

    WHERE   
        (bo.end_date >= CURRENT_dATE or bo.end_date IS NULL)
        AND bo.start_date <= CURRENT_DATE
        ---TIME VALIDATIONS
        -------------------
        AND bo.offer_discount_type = 'PERCENT_OFF'
        AND bo.offer_type = 'ORDER_ITEM'
        AND bo.sndbx_id is NULL
        AND (bo.archived is NULL OR bo.archived='N')
        AND (bo.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
        AND bo.automatically_added = 'true'
        AND (boic.sndbx_tier is NULL OR bo.sndbx_tier = 999999)
        AND (bocsx.sndbx_tier is NULL OR bocsx.sndbx_tier = 999999)
        --AND bocsx.customer_segment_id IS NOT NULL
        AND bs.site_identifier_value = 'BOG'
        AND (bo.max_uses_per_customer <> 1 OR bo.max_uses_per_customer IS NULL)
    ),
    
    tab_unnest AS (
        SELECT 1 AS n UNION ALL
        SELECT 2 AS n UNION ALL
        SELECT 3 AS n UNION ALL
        SELECT 4 AS n UNION ALL
        SELECT 5 AS n UNION ALL
        SELECT 6 AS n UNION ALL
        SELECT 7 AS n UNION ALL
        SELECT 8 AS n UNION ALL
        SELECT 9 AS n UNION ALL
        SELECT 10 AS n UNION ALL
        SELECT 11 AS n UNION ALL
        SELECT 12 AS n UNION ALL
        SELECT 13 AS n UNION ALL
        SELECT 14 AS n UNION ALL
        SELECT 15 AS n UNION ALL
        SELECT 16 AS n UNION ALL
        SELECT 17 AS n UNION ALL
        SELECT 18 AS n UNION ALL
        SELECT 19 AS n UNION ALL
        SELECT 20 AS n UNION ALL
        SELECT 21 AS n UNION ALL
        SELECT 22 AS n UNION ALL
        SELECT 23 AS n UNION ALL
        SELECT 24 AS n UNION ALL
        SELECT 25 AS n UNION ALL
        SELECT 26 AS n UNION ALL
        SELECT 27 AS n UNION ALL
        SELECT 28 AS n UNION ALL
        SELECT 29 AS n UNION ALL
        SELECT 30 AS n UNION ALL
        SELECT 31 AS n UNION ALL
        SELECT 32 AS n UNION ALL
        SELECT 33 AS n UNION ALL
        SELECT 34 AS n UNION ALL
        SELECT 35 AS n UNION ALL
        SELECT 36 AS n UNION ALL
        SELECT 37 AS n UNION ALL
        SELECT 38 AS n UNION ALL
        SELECT 39 AS n UNION ALL
        SELECT 40 AS n UNION ALL
        SELECT 41 AS n UNION ALL
        SELECT 42 AS n UNION ALL
        SELECT 43 AS n UNION ALL
        SELECT 44 AS n UNION ALL
        SELECT 45 AS n UNION ALL
        SELECT 46 AS n UNION ALL
        SELECT 47 AS n UNION ALL
        SELECT 48 AS n UNION ALL
        SELECT 49 AS n UNION ALL
        SELECT 50 AS n UNION ALL
        SELECT 51 AS n UNION ALL
        SELECT 52 AS n UNION ALL
        SELECT 53 AS n UNION ALL
        SELECT 54 AS n UNION ALL
        SELECT 55 AS n UNION ALL
        SELECT 56 AS n UNION ALL
        SELECT 57 AS n UNION ALL
        SELECT 58 AS n UNION ALL
        SELECT 59 AS n UNION ALL
        SELECT 60 AS n UNION ALL
        SELECT 61 AS n UNION ALL
        SELECT 62 AS n UNION ALL
        SELECT 63 AS n UNION ALL
        SELECT 64 AS n UNION ALL
        SELECT 65 AS n UNION ALL
        SELECT 66 AS n UNION ALL
        SELECT 67 AS n UNION ALL
        SELECT 68 AS n UNION ALL
        SELECT 69 AS n UNION ALL
        SELECT 70 AS n UNION ALL
        SELECT 71 AS n UNION ALL
        SELECT 72 AS n UNION ALL
        SELECT 73 AS n UNION ALL
        SELECT 74 AS n UNION ALL
        SELECT 75 AS n UNION ALL
        SELECT 76 AS n UNION ALL
        SELECT 77 AS n UNION ALL
        SELECT 78 AS n UNION ALL
        SELECT 79 AS n UNION ALL
        SELECT 80 AS n UNION ALL
        SELECT 81 AS n UNION ALL
        SELECT 82 AS n UNION ALL
        SELECT 83 AS n UNION ALL
        SELECT 84 AS n UNION ALL
        SELECT 85 AS n UNION ALL
        SELECT 86 AS n UNION ALL
        SELECT 87 AS n UNION ALL
        SELECT 88 AS n UNION ALL
        SELECT 89 AS n UNION ALL
        SELECT 90 AS n UNION ALL
        SELECT 91 AS n UNION ALL
        SELECT 92 AS n UNION ALL
        SELECT 93 AS n UNION ALL
        SELECT 94 AS n UNION ALL
        SELECT 95 AS n UNION ALL
        SELECT 96 AS n UNION ALL
        SELECT 97 AS n UNION ALL
        SELECT 98 AS n UNION ALL
        SELECT 99 AS n UNION ALL
        SELECT 100 AS n UNION ALL
        SELECT 101 AS n UNION ALL
        SELECT 102 AS n UNION ALL
        SELECT 103 AS n UNION ALL
        SELECT 104 AS n UNION ALL
        SELECT 105 AS n UNION ALL
        SELECT 106 AS n UNION ALL
        SELECT 107 AS n UNION ALL
        SELECT 108 AS n UNION ALL
        SELECT 109 AS n UNION ALL
        SELECT 110 AS n UNION ALL
        SELECT 111 AS n UNION ALL
        SELECT 112 AS n UNION ALL
        SELECT 113 AS n UNION ALL
        SELECT 114 AS n UNION ALL
        SELECT 115 AS n UNION ALL
        SELECT 116 AS n UNION ALL
        SELECT 117 AS n UNION ALL
        SELECT 118 AS n UNION ALL
        SELECT 119 AS n UNION ALL
        SELECT 120 AS n
    ),
     
    info_skus AS (  
    SELECT 
        bs.offer_id,
        bs.dct,
        bs.customer_segment_id,
        TRIM(SPLIT_PART(bs.skus, ',', tab_unnest.n)) AS sku
        
    FROM tab_unnest
    INNER JOIN base_skus bs ON tab_unnest.n <= REGEXP_COUNT(bs.skus, ',') + 1
    )
    
SELECT DISTINCT
    s.site_identifier_value as ciudad,
    --bcs.external_identifier::int AS customer_id,
    bs.addl_product_id AS card_id,
    MAX(bo.dct) AS dct
    
FROM info_skus bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bo.sku
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_customer_customer_seg_xref"   bcs     ON bo.customer_segment_id = bcs.customer_segment_id AND bo.customer_segment_id IS NOT NULL AND bcs.external_identifier ~ '^[0-9]+$'
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON s.site_id = bs.catalog_disc

WHERE bo.customer_segment_id IS NULL
GROUP BY 1,2--,3
"""
df_no_filt_offers = read_connection_data_warehouse.runQuery(query)
df_no_filt_offers[["dct"]] = df_no_filt_offers[["dct"]].astype(float)

In [13]:
query = """
SELECT DISTINCT 
region,
sku,
CAST(day AS date) AS day,
MAX(CASE WHEN account = 'GMV Resale Subtotal Acumulado' THEN value END) AS gmv_value,
MAX(CASE WHEN account = 'COGS Acumulado' THEN value END) AS cogs_value,
(MAX(CASE WHEN account = 'GMV Resale Subtotal Acumulado' THEN value END) - MAX(CASE WHEN account = 'COGS Acumulado' THEN value END)) / MAX(CASE WHEN account = 'GMV Resale Subtotal Acumulado' THEN value END) AS margin

FROM mvp_operations.raw_data_aleph

WHERE (CAST(day AS date) = DATEADD(day, -1, DATEADD(month, 1, DATE_TRUNC('month', CAST(day AS date)))) OR CAST(day AS date) = DATEADD(day, -1, CURRENT_DATE) OR CAST(day AS date) = '2022-12-14' OR CAST(day AS date) = '2023-03-18')
AND CAST(day AS date) > DATE_TRUNC('month', date(getdate()) - interval '0 month')
AND value > 1

GROUP BY 1,2,3
"""
df_margen = run_read_dwd_query(query)
df_margen[["margin"]] = df_margen[["margin"]].astype(float)

In [14]:
query = """
SELECT DISTINCT 
  p.region_code,
  p.name AS sku_name,
  bs2.name AS card_name,
  bs.addl_product_id AS card_id,
  p.product_id AS product_id,
  bs.sku_id,
  p.sku,
  bcat2.name as cat,
  bcat.name as subcat
FROM postgres_main_co."purchase_orders.products"                        p
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"              bs      ON bs.upc = p.sku
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_product"           bp      ON bp.product_id = bs.addl_product_id --Conectar sku con la tarjeta
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_sku"               bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"     bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL --Relaciones categorias 
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"          bcat    ON bp.default_category_id = bcat.category_id --Nombre subcategoria
LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_category"          bcat2   ON bcx.category_id = bcat2.category_id --Nombre categoria

where
  p.deleted_at is null
  and p.parent_id is null
  and bs.archived = 'N'
  and bs.sndbx_id is null
  and bp.archived = 'N'
  and bs.catalog_disc < 0
  and ((bp.archived = 'N' and bp.sndbx_id is null) or (bp.archived is null))
  """
df_aux1 = read_connection_data_warehouse.runQuery(query)

In [15]:
df_margen = pd.merge(df_margen[['region','sku','margin']], df_aux1[['sku','card_name','card_id']],  how='inner', left_on=['sku'], right_on = ['sku'])
df_margen = df_margen.dropna()
df_margen = df_margen.groupby(['region','card_id','card_name']).margin.mean().reset_index().copy()

In [22]:
query = """
SELECT 
s.identifier_value AS city,
DATE(fs.order_submitted_date) AS submit_date,
fs.order_id,
dc.source_id AS customer_id,
cat.parent_description as category,
cat.description AS subcategory,
dp.card_id,
dp.card_description AS product_name,
fs.product_quantity_x_step_unit AS cant,
CASE
    WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fs.gmv_pxq_local/4.75
    WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fs.gmv_pxq_local/3776
    WHEN s.identifier_value IN ('CMX') THEN fs.gmv_pxq_local/19.65
    ELSE fs.gmv_pxq_local
END AS gmv_usd,
CASE
    WHEN s.identifier_value IN ('SPO','BHZ','CWB','VCP') THEN fsd.product_discount/4.75
    WHEN s.identifier_value IN ('BOG','BAQ','MDE') THEN fsd.product_discount/3776
    WHEN s.identifier_value IN ('CMX') THEN fsd.product_discount/19.65
    ELSE fsd.product_discount
END AS discount_applied
FROM dpr_sales.fact_sales                   fs
INNER JOIN dpr_shared.dim_customer          dc  ON dc.customer_id = fs.dim_customer
INNER JOIN dpr_shared.dim_site              s   ON s.site_id = fs.dim_site
INNER JOIN dpr_shared.dim_product           dp  ON dp.product_id = fs.dim_product
INNER JOIN dpr_shared.dim_category          cat ON cat.category_id = dp.category_id
LEFT JOIN dpr_sales.fact_sales_discounts    fsd ON fs.order_item_id = fsd.order_item_id 
WHERE fs.gmv_enabled = TRUE
AND fulfillment_order_status NOT IN ('CANCELLED', 'ARCHIVED','No value')
AND fs.fb_order_status_id  IN (1,6,7,8)
AND fs.is_deleted = FALSE
AND fs.dim_status = 1
AND dp.is_slot = 'false'
AND s.identifier_value IN ('BOG','BAQ','MDE','CMX','SPO','BHZ','CWB','VCP')
--AND s.identifier_value = '{city}'
AND dc.source_id IN {clientes}
""".format(clientes=tuple(df_kams_ctm.customer_id.unique()), city = city)

data_ventas_aux = run_read_dwd_query(query)
data_ventas_aux[["gmv_usd", "cant","discount_applied"]] = data_ventas_aux[["gmv_usd", "cant","discount_applied"]].astype(float)
data_ventas_aux['submit_date'] = pd.to_datetime(data_ventas_aux['submit_date']) 

In [23]:
df = data_ventas_aux.loc[data_ventas_aux.card_id.isin(df_skus.card_id.unique())].groupby(by=['city','customer_id','submit_date','category','subcategory','card_id','product_name']).agg({'gmv_usd':'sum','cant':'sum','discount_applied':'sum'}).reset_index()

In [24]:
#this is for the whole kam behavior
df_all = df.copy()

# Generacion de df con info by product by client

In [25]:
df = df.sort_values('submit_date',ignore_index=True)

#fix for id problems and taking into account only dates
df['order_id']= df.submit_date.apply(lambda x:x.toordinal()) 

#Days of difference between orders for customer and for cat
df['days_bet'] = (df.submit_date -df.groupby('customer_id').submit_date.transform('min')).dt.days

In [26]:
#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df['res'] = df.groupby(by=['customer_id', 'card_id', 'product_name'])['days_bet'].diff()
df_grouped = df.groupby(by=['city', 'customer_id', 'card_id', 'product_name']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                                    'order_id': [lambda x: x.nunique()], 
                                                                                    'gmv_usd': [np.sum, np.mean],
                                                                                    'discount_applied': [np.sum, np.mean],
                                                                                    'cant': [np.sum, np.mean],
                                                                                    'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                                    }).reset_index()

In [27]:
#Finally we define everything in a df2 for further processing
df2 = df_grouped.copy()

df2.columns = ['city', 'customer_id', 'card_id', 'product_name', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'gmv_cat_usd','aov_cat_usd', 'all_dct_usd', 'avg_dct_usd','all_cant_cat','avg_cant_cat', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [28]:
df2 = pd.merge(df2, df_margen[['card_id','margin']],  how='left', left_on=['card_id'], right_on = ['card_id'])

In [29]:
df2['margin'].fillna(0.1, inplace=True)
df2['avg_cash_margin'] = df2.aov_cat_usd * df2.margin
df2['all_cash_margin'] = df2.gmv_cat_usd * df2.margin
df2['avg_net_cash_margin'] = df2.avg_cash_margin - df2.avg_dct_usd
df2['all_net_cash_margin'] = df2.all_cash_margin - df2.all_dct_usd

In [30]:
df2["churned"] = [0 if ((x[18]) - (x[6]+x[8]*2) <= 0) else 1 for x in df2.values]

In [31]:
df2['churned_date'] = df2.apply(lambda x: (x[16] + datetime.timedelta(days=int(x[6]) + int(x[8]) * 2)).strftime('%Y-%m-%d') if not (pd.isnull(x[6]) or pd.isnull(x[8])) else None, axis=1)
df2['churned_date'] = pd.to_datetime(df2['churned_date'], errors='coerce')

# Genracion df y info by customer

In [32]:
df_all = pd.merge(df_all, df_margen[['card_id','margin']],  how='left', left_on=['card_id'], right_on = ['card_id'])

In [33]:
df_all['margin'].fillna(0.05, inplace=True)
df_all['cash_margin'] = df_all.gmv_usd * df_all.margin
df_all['net_cash_margin'] = df_all.cash_margin - df_all.discount_applied

In [34]:
df_all = df_all.sort_values('submit_date',ignore_index=True)

#fix for id problems and taking into account only dates
df_all['order_id']= df_all.submit_date.apply(lambda x:x.toordinal()) 

#Days of difference between orders for customer and for cat
df_all['days_bet'] = (df_all.submit_date -df_all.groupby('customer_id').submit_date.transform('min')).dt.days

In [35]:
#This calculate in res the difference between consecutive orders and with the agg we calculate all metrics
df_all['res'] = df_all.groupby(by=['customer_id', 'card_id', 'product_name'])['days_bet'].diff()
df_grouped_all = df_all.groupby(by=['city', 'customer_id']).agg({'res': [np.min, np.max, np.mean, np.median, np.std],
                                                                 'order_id': [lambda x: x.nunique()], 
                                                                 'gmv_usd': [np.sum, np.mean],
                                                                 'discount_applied': [np.sum, np.mean],
                                                                 'cash_margin': [np.sum, np.mean],
                                                                 'cant': [np.sum, np.mean],
                                                                 'submit_date': [np.max, lambda x: (x.max()-x.min()).days, lambda x: (datetime.datetime.today() - x.max()).days, np.min]
                                                                 }).reset_index()

In [36]:
#Finally we define everything in a df2 for further processing
df2_all = df_grouped_all.copy()

df2_all.columns = ['city', 'customer_id', 'min_diff_days', 'max_diff_days', 'avg_diff_days', 'median_diff_days', 'stdev_diff_days',
               'num_orders', 'gmv_cat_usd','aov_cat_usd', 'all_dct_usd', 'avg_dct_usd', 'all_cash_margin', 'avg_cash_margin','all_cant_cat','avg_cant_cat', 'last_order_date', 'diff_from_last_to_first_order', 'days_since_last_order','first_order_date']

In [37]:
df2_all['adjust_std'] = pd.qcut(df2_all.stdev_diff_days, 10, labels=[1,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.2,0.1])
df2_all[["adjust_std"]] = df2_all[["adjust_std"]].astype(float)

In [38]:
df2_all["cltv"] = 1000.00 * df2_all.all_cash_margin * df2_all.adjust_std * df2_all.num_orders / df2_all.diff_from_last_to_first_order
df2_all['cltv'].fillna(0, inplace=True)

In [39]:
# Define the percentiles for tiers
percentiles = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
tier_labels = ['E', 'D', 'C', 'B', 'A']

# Add a new column with the tier labels based on percentiles
df2_all['tier'] = df2_all.loc[df2_all.city.isin(['BOG','BAQ','MDE','CMX','SPO','BHZ','CWB','VCP'])].groupby('city')['cltv'].transform(lambda x: pd.qcut(x, q=percentiles, labels=tier_labels))

In [40]:
df2_all["all_churned"] = [0 if ((x[18]) - (x[4]+x[6]*2) <= 0) else 1 for x in df2_all.values]
df2_all['all_churned_date'] = df2_all.apply(lambda x: (x[16] + datetime.timedelta(days=int(x[4]) + int(x[6]) * 2)).strftime('%Y-%m-%d') if not (pd.isnull(x[4]) or pd.isnull(x[6])) else None, axis=1)
df2_all['all_churned_date'] = pd.to_datetime(df2_all['all_churned_date'], errors='coerce')

# Merge all info 

In [41]:
#BRING NAME AND RESPONSABLE OF THE KAM
df3 = pd.merge(df2, df_kams_ctm[['customer_id','user_name','kam_name','status']],  how='left', left_on=['customer_id'], right_on = ['customer_id'])

In [42]:
#BRING CHURN ALL DATE OF THE KAM
df3 = pd.merge(df3, df2_all[['customer_id','all_churned','all_churned_date','cltv','tier']],  how='left', left_on=['customer_id'], right_on = ['customer_id'])

In [43]:
#BRING CATEGORY AND STATUS FRIDA
df3 = pd.merge(df3, df_skus[['card_id','categoria','subcategoria','status_frida']],  how='left', left_on=['card_id'], right_on = ['card_id'])

df3[['categoria']] = df3[['categoria']].fillna('DELETED')
df3[['subcategoria']] = df3[['subcategoria']].fillna('DELETED')
df3[['status_frida']] = df3[['status_frida']].fillna('DELETED')

In [44]:
df3 = pd.merge(df3, df_prices,  how='left', left_on=['card_id'], right_on = ['card_id'])
df3 = pd.merge(df3, df_no_filt_offers[['card_id','dct']],  how='left', left_on=['card_id'], right_on = ['card_id'])
df3 = pd.merge(df3, df_filt_offers[['customer_id','card_id','dct']],  how='left', left_on=['customer_id','card_id'], right_on = ['customer_id','card_id'])

In [45]:
df3['dct_x'].fillna(0, inplace=True)
df3['dct_y'].fillna(0, inplace=True)

In [46]:
df3['net_price_all'] = (df3.gross_price_no_iva * (1-df3.dct_x)) + (df3.gross_price - df3.gross_price_no_iva)
df3['net_price_spfc'] = (df3.gross_price_no_iva * (1-df3.dct_y)) + (df3.gross_price - df3.gross_price_no_iva)

df3['gpi'] = 100.00 * df3.gross_price / df3.benchmark
df3['npi_all'] = 100.00 * df3.net_price_all / df3.benchmark
df3['npi_spfc'] = 100.00 * df3.net_price_spfc / df3.benchmark

In [47]:
df3['best_net_price'] = np.maximum(df3['net_price_all'].fillna(-1), df3['net_price_spfc'].fillna(-1))
df3['best_npi'] = np.maximum(df3['npi_all'].fillna(-1), df3['npi_spfc'].fillna(-1))

In [48]:
# Get the current date
current_date = datetime.datetime.now()

# Calculate the start and end dates of the current week
start_of_week = current_date - timedelta(days=current_date.weekday())
end_of_week = start_of_week + timedelta(days=6)
current_date2 = datetime.datetime.now().strftime("%d-%m-%Y")

In [49]:
# Define the current date, last month, and the month before last
last_month = current_date - timedelta(days=30)
month_before_last = current_date - timedelta(days=60)  # Two months before

In [50]:
df3 = df3.loc[(df3.benchmark>0) & (df3.status_frida == 'on') & (df3.churned_date >= month_before_last) & (df3.churned_date <= current_date)].reset_index(drop=True)

In [51]:
condition_city = df3.city.isin(['CMX', 'BAQ', 'BOG', 'MDE'])

In [52]:
# Define the conditions for both cases
condition_with_100 = (df3.gpi - df3.margin * 100.00 * 0.85 < 95) & \
                     (df3.best_npi > 95)

condition_with_95 = (df3.gpi - df3.margin * 100.00 * 0.85 < 95) & \
                    (df3.best_npi > 95)
                    
# Apply the conditions using np.where and filter df_dct
df_dct = df3.copy()
df_dct['condition'] = np.where(condition_city, condition_with_100, condition_with_95)
df_dct = df_dct[df_dct['condition']].copy()
df_dct.drop(columns=['condition'], inplace=True)

condition_city = df_dct.city.isin(['CMX', 'BAQ', 'BOG', 'MDE'])
# Calculate the 'dct' column conditionally
df_dct['dct'] = np.where(condition_city,  # Apply the condition to df_dct
                         np.round((1 - (df_dct.benchmark * 0.95 / df_dct.gross_price)) * 100.00, 2),
                         np.round((1 - (df_dct.benchmark * 0.95 / df_dct.gross_price)) * 100.00, 2))

In [53]:
#df_dct['dct'] = np.round((1 - (df_dct.benchmark * 0.99 / df_dct.gross_price)) * 100.00,2)
df_dct['proportion_dct'] = np.round(df_dct.dct_x / (df_dct.dct/100.00),2)  

In [54]:
df_dct = df_dct.loc[(df_dct.proportion_dct < 1)]

In [55]:
df_groups = df.loc[(df.customer_id.isin(df_dct.customer_id.unique())) & (df.card_id.isin(df_dct.card_id.unique()))].groupby(["city","customer_id"]).agg({'gmv_usd':np.sum,'card_id': lambda x: x.nunique()}).reset_index()

In [56]:
from sklearn.model_selection import train_test_split

def create_groups(offer):
    customers = offer.customer_id.unique()
    
    alpha = 0.01
    aux = 0
    
    while True:
        customer_control, customer_test = train_test_split(customers, test_size=0.50)
        
        avg_gmv_control = offer[offer.customer_id.isin(customer_control)]['gmv_usd'].mean()
        avg_gmv_test = offer[offer.customer_id.isin(customer_test)]['gmv_usd'].mean()
        
        avg_products_control = offer[offer.customer_id.isin(customer_control)]['card_id'].mean()
        avg_products_test = offer[offer.customer_id.isin(customer_test)]['card_id'].mean()
        
        # Check if both average GMV and average products meet the criteria
        if (
            abs(1 - (avg_gmv_control / avg_gmv_test)) <= alpha and
            abs(1 - (avg_products_control / avg_products_test)) <= alpha
        ):
            break
        
        aux += 1
        
        if aux == 1000:
            alpha += 0.01
            aux = 0
    print(alpha)
    return customer_control.tolist(), customer_test.tolist()


In [57]:
result_rows = []
for city in df_groups.city.unique():
    control, piloto = create_groups(df_groups.loc[df_groups.city == city])
    
    for customer_id in control:
        result_rows.append([city, customer_id, 'control'])
    
    for customer_id in piloto:
        result_rows.append([city, customer_id, 'piloto'])

result_df = pd.DataFrame(result_rows, columns=['city', 'customer_id', 'group'])

0.01
0.01
0.01
0.01
0.01
0.02
0.01
0.01


In [58]:
df_dct2 = pd.merge(df_dct, result_df,  how='left', left_on=['city','customer_id'], right_on = ['city','customer_id'])
df_dct3 = pd.merge(df_dct, result_df,  how='left', left_on=['city','customer_id'], right_on = ['city','customer_id'])

In [59]:
df_dct2 = df_dct2.loc[df_dct2.group == 'piloto']

In [60]:
df_dct2["offer_name"] = (
    "KAM_PTOOL_EXP.DCT_" +
    df_dct2["city"] + "_" +
    df_dct2["product_name"].str.replace(",", ".") + "_" + current_date2 + "_" +
    df_dct2["dct"].astype(str) + "%_HDCT_" + 
    df_dct2["proportion_dct"].astype(str)
)

In [61]:
def segments(offer, cityy):
        import requests
        import json
        
        global_segment_list = [] #//for procesing
        for i in offer.customer_id.unique():
                segment = {'name':str(i), 'customersIds': [int(i)],"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(7))).strftime('%Y-%m-%d')}
                global_segment_list.append(segment) 
        
        url = f"https://{cityy}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

        payload = json.dumps(global_segment_list)


        headers = {
        'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
        'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)
        
        sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]
        ssgtt = pd.DataFrame(sgt, columns=['segment_id', 'customer_id'])
        ssgtt[["customer_id"]] = ssgtt[["customer_id"]].astype(int)
        
        return ssgtt                

In [69]:
def csv_files(offer,segment):
    csv_file = {} 
    csv_file['offer_name'] = offer.offer_name
    csv_file['offer_description'] = None
    csv_file['discount'] = offer.dct
    csv_file['automatically_consider_offer'] = "true"
    csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=0)).strftime('%d-%m-%Y')
    csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(7))).strftime('%d-%m-%Y')
    csv_file['max_uses_per_order'] = 0
    csv_file['max_uses_per_customer'] = 2
    csv_file['customer_segment_id'] = [segment.loc[segment.customer_id == customer].segment_id.unique()[0] for customer in offer.customer_id]

    sku_ids = []
    for sku in offer.card_id:
        try:
            sku_id = frida_products.loc[frida_products.id_tarjeta == sku, "skus"].values[0]
        except IndexError:
            sku_id = None  # Assign a null value if IndexError occurs
        sku_ids.append(sku_id)
    
    csv_file['sku_id'] = sku_ids
    
    df = pd.DataFrame(csv_file)
    
    # Drop rows with null values in the 'sku_id' column
    df = df.dropna(subset=['sku_id'])
    return df

In [63]:
df_dct3.to_csv(f"grupos_KAM_{current_date2}.csv", index = False)

In [70]:
for city in df_dct2.city.unique():
    print(city)
    
    segment = segments(df_dct2.loc[df_dct2.city == city],city)
    
    city_csv = csv_files(df_dct2.loc[df_dct2.city == city],segment)
    city_csv = city_csv.drop_duplicates()
    city_csv.to_csv(f"{city}_KAM_EXP.DCT{current_date2}.csv", index = False)

BAQ
BHZ
BOG
CMX
CWB
MDE
SPO
VCP
